In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from mediawiki import MediaWiki
from bs4 import BeautifulSoup
import string
from nltk import tokenize
from nltk.tag import StanfordNERTagger
import string
from nltk.tokenize import word_tokenize
import unicodedata
import timeit

In [2]:
# Using the Stanfors' default tagger
st = StanfordNERTagger('/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz',
  '/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/stanford-ner.jar',
   encoding='utf-8') 

In [3]:
# Lists containing common prefixes and suffixes.
prefix = ['mr','by','mrs','miss','dr', 'president','prime minister']
# https://www.irfca.org/docs/place-names.html
#suffixes = ['nagar','colony','street','road','hill','river','temple']
#subs = ['pur','puram','ur','ghar','pura','ganj','abad','halli','keri']

In [4]:
#news_data = pd.read_csv('/Users/venkatakrishnamohansunkara/Desktop/DM/GDELT_2017/large_dataset/unrest_2017_ie_toi.csv', sep=',')
df = pd.read_csv('../document similarity/document_similarity_hatt.csv')

In [5]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,category,content,doc2vec,doc2veccosine,HATT embeddings
0,0,0,0,movie,Padmaavat: Why a Bollywood epic has sparked fi...,[ -9.05430984 1.71762216 1.08929253 -3.22...,[ 1. 0.94052142 0.9796434 0.926054...,[-0.04064541 0.5168978 -0.10278964 -0.406168...
1,1,1,1,lokpal,Anna Hazare on fast-unto-death demanding Jan L...,[ -4.08943748e+00 3.65195364e-01 4.3100047...,[ 0.94052142 1. 0.97000271 0.940714...,[ 1.61063448e-01 5.65693378e-01 -4.65846695e-...
2,2,2,2,movie,A Film Has Inflamed Indians. But Moviegoers Mo...,[ -8.49335098 0.55525446 1.04886711 -3.75...,[ 0.9796434 0.97000271 1. 0.946262...,[ 1.81895524e-01 5.93335807e-01 6.25734329e-...
3,3,3,3,movie,Indian police detain dozens protesting against...,[ -6.10597134 0.87642241 0.60653818 -3.09...,[ 0.92605472 0.9407143 0.94626272 0.999999...,[ 0.17177399 0.6309512 -0.04849247 -0.051712...
4,4,4,4,lokpal,Thousands Back Antigraft Hunger Strike in New ...,[-11.33139229 2.65446663 1.2543447 -0.94...,[ 0.8824504 0.86918712 0.90175384 0.953397...,[-0.06123626 -1.1703769 0.01185594 -0.637607...
5,5,5,5,lokpal,Jan Lokpal Bill: Protests continue in support ...,[ -4.41828299e+00 5.35881042e-01 6.4218103...,[ 0.93510097 0.97717178 0.95960432 0.905399...,[-3.85057926e-02 3.16520661e-01 -2.69440532e-...
6,6,6,6,lokpal,India corruption: Protests swell in support of...,[ -7.38126898 0.73874789 1.25690413 -4.65...,[ 0.97860324 0.96208191 0.97947335 0.944147...,[ 1.35564059e-01 5.99828005e-01 -5.64954542e-...
7,7,7,7,lokpal,Hunger strike over Lokpal Bill as thousands pr...,[-5.09889364 0.64079404 0.13031445 -4.445450...,[ 0.95688981 0.98412955 0.96912926 0.928257...,[ 1.44097835e-01 4.18904036e-01 -1.80478878e-...
8,8,8,8,movie,Padmaavat: Riots break out in India as protest...,[ -2.40009403e+00 6.57581329e-01 4.1137728...,[ 0.88772088 0.95705724 0.91422349 0.880223...,[ 0.03712982 0.63247323 -0.16278782 -0.318585...


In [72]:
test_data = df.iloc[1].content

In [5]:
# Function for removing unnecessary characters.
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", " ", string)
    string = re.sub(r"\'", " ", string)
    string = re.sub(r"\"", " ", string)
    string = re.sub(r"\n", " ", string)
    string = re.sub(r"=", " ", string)
    string = re.sub(r"-", " ", string)
    string = re.sub(r"/", " ", string)
    return string.strip()

In [11]:
test_data = "Delhi Congress to protest on Saturday against PM Modi's demonetisation. Congress on Monday said it will stage protests across Delhi on Saturday as part of a pan-India stir against demonetisation and will seek answers from Prime Minister Narendra Modi on the amount of black money unearthed. Delhi Congress chief spokesperson Sharmistha Mukherjee said people were waiting for Prime Minister to give details of the black money unearthed due to the massive exercise during his address to the nation on New Year Eve, but he kept mum. She claimed 115 people died during the 50 days Modi had sought to stabilise the situation after his sudden announcement on November 8 to demonetise high-value bank notes had triggered.Watch What Else Is Making NewsPeople expected Modi to announce compensation to affected families, but he did not mention them in his address, said Mukherjee. She also repeated Rahul Gandhis charges that Modi as Gujarat Chief Minister had taken money from Sahara and Birla groups. Modi has not yet replied to Congress vice-president Rahul Gandhis poser about receiving Rs 65 crore from Aditya Birla Group and Sahara Group in a blatant case of corruption, she said.During the protests in all the 14 districts in Delhi, party leaders and workers will seek answers on the questions and demand disclosure of the amount of black money unearthed, 20 per cent bonus on minimum support price to farmers, disclosure of names of people who deposited Rs 25 lakh or more prior to demonetisation. She said it was very strange that during demonetisation BJP leaders were caught with large amount of old and new currency notes, and chanrged that they had prior information of the governments move.BJP leaders had knowledge about demonetisation beforehand and thats why crores of rupees were invested in land and property by RSS and BJP leaders in Bengal, Orissa and Bihar, she alleged"

In [12]:
test_data = unicodedata.normalize("NFKD", test_data)
test_data = clean_str(test_data)

In [13]:
import en_coref_lg
nlp = en_coref_lg.load()

In [14]:
doc = nlp(test_data)

In [15]:
test_data = doc._.coref_resolved

In [78]:
doc._.coref_clusters[7]

the villagers: [the villagers, their, their]

In [16]:
test_data

'Delhi Congress to protest on Saturday against PM Modi s demonetisation. Delhi Congress on Monday said Delhi Congress will stage protests across Delhi on Saturday as part of a pan India stir against demonetisation and will seek answers from Prime Minister Narendra Modi on the amount of black money unearthed. Delhi Congress Delhi Congress chief spokesperson Sharmistha Mukherjee said people were waiting for Prime Minister to give details of the black money unearthed due to the massive exercise during Delhi Congress chief spokesperson Sharmistha Mukherjee address to the nation on New Year Eve, but Delhi Congress chief spokesperson Sharmistha Mukherjee kept mum. Delhi Congress chief spokesperson Sharmistha Mukherjee claimed 115 people died during the 50 days Modi had sought to stabilise the situation after Modi sudden announcement on November 8 to demonetise high value bank notes had triggered.Watch What Else Is Making NewsPeople expected Modi to announce compensation to affected families,

In [80]:
start = timeit.default_timer()
# Convert the entire document data to sentences.
text_sent = nltk.sent_tokenize(test_data)
stanford = []
noun_phrases = []
prp_phrases = []
#new_model = []
whos = []
# other are prefixes
other = []
#suffix_locations = []
#sub_locations = []
all_words = []
# Iterate over all the sentences.
sent_no=0;
total_words = 0;
for sent in text_sent:
    # Convert the sentence to words
    text_tags = nltk.word_tokenize(sent)
    curr_words = len(text_tags)
    all_words+=[text_tags]
    # Get the NER for the words.
    original_tags = st.tag(text_tags)
    l = len(original_tags)
    i=0;
    # Iterate over the tagged words.
    while i<l:
        #print(i)
        e,t = original_tags[i];
        ci = i;
        # If it's a location, then check the next 3 words.
        if t == 'PERSON' or t == 'ORGANIZATION':
            j = 1;
            s = e; 
            # Verify the tags for the next 3 words.
            while i+j<len(original_tags):
                # If the next words are also locations, then concatenate them to make a large string.
                if original_tags[i+j][1] == 'PERSON' or original_tags[i+j][1] == 'ORGANIZATION':
                    s = s+" "+original_tags[i+j][0];
                    j+=1;
                else:
                    break;
            i = i+j;
            # Save the candidates to a whos list
            whos+=[(sent_no,total_words+ci,s,ci)];
        else:
            i=i+1;
    # Get the POS tag for the words.
    text_tagged = nltk.pos_tag(text_tags)
    l = len(text_tagged)
    i=0;
    # iterate over the tagged words.
    while i<l:
        #print(i)
        e,t = text_tagged[i];
        # If the current word is a Noun phrase.
        if t == 'NNP':
            j = 1;
            s = e;
            # verify the tags for the next 3 words.
            while i+j<len(text_tagged):
                # If the next words are also nouns, then concatenate them to make a large string.
                if text_tagged[i+j][1] in ['NNP','NN','NNS','NNPS']:
                    s = s+" "+text_tagged[i+j][0];
                    j+=1;
                else:
                    break;
            i = i+j;
            # Save the noun phrases to a noun_phrases list
            noun_phrases+=[s];
        elif t == 'PRP' or t == 'PRP$' or t == 'WP' or t == 'WP$':
            j = 1;
            s = e;
            # verify the tags for the next 3 words.
            while i+j<len(text_tagged):
                # If the next words are also nouns, then concatenate them to make a large string.
                if text_tagged[i+j][1] in ['PRP','PRP$','WP','WP$']:
                    s = s+" "+text_tagged[i+j][0];
                    j+=1;
                else:
                    break;
            i = i+j;
            # Save the noun phrases to a noun_phrases list
            prp_phrases+=[s];
        else:
            i=i+1;
    # Convert the words to lower case so as to compare with the prefixes.
    text_tags = [t.lower() for t in text_tags]
    # Go through all the prefixes.
    for k in prefix:
        # If the prefix is not present in the sentence then an error occurs. So, we use a try, catch block
        try:
            # Get the position the prefix word in a sentence.
            index = text_tags.index(k)
            # If the word after the prefix word is a noun, then consider it.
            if text_tagged[index+1][1] == 'NNP':
                j = 1;
                s = text_tagged[index+1][0]; 
                # Verify if it is a phrase by considering the next 3 words.
                while index+j<len(text_tagged):
                    if text_tagged[index+j+1][1] in ['NNP','NN','NNS','NNPS']:
                        s = s+" "+text_tagged[index+j+1][0];
                        j+=1;
                    else:
                        break;
                # Save the words after the prefix words in to other list.
                other = other + [(sent_no,index+1,s)]
        except:
            continue
    sent_no+=1;
    total_words+=curr_words
stop = timeit.default_timer()
print('Time: ', stop - start) 
all_words = [x for sublist in all_words for x in sublist]
#only by stanford ner
all_whos = whos

Time:  36.32420918304706


In [81]:
who_candidates = all_whos

In [82]:
who_candidates

[(0, 0, 'Anna Hazare', 0),
 (0, 7, 'Jan Lokpal Bill Veteran Gandhian', 7),
 (0, 14, 'Anna Hazare', 14),
 (0, 32, 'Jan Lokpal Bill', 32),
 (1, 62, 'Mahatma Gandhi', 23),
 (2, 76, 'Swami Agnivesh', 9),
 (2, 85, 'Kiran Bedi', 18),
 (2, 92, 'Sandeep Pandey', 25),
 (3, 125, 'Jan Lokpal Bill', 30),
 (3, 131, 'Hazare', 36),
 (4, 148, 'Hazare', 12),
 (5, 168, 'Prime Minister s Office', 5),
 (5, 184, 'Hazare', 21),
 (6, 203, 'Hazare', 11),
 (7, 234, 'Hazare', 24),
 (8, 237, 'Hazare', 1),
 (8, 240, 'Hazare', 4),
 (9, 285, 'Bill', 7),
 (9, 296, 'Hazare', 18),
 (10, 300, 'Hazare', 1),
 (10, 314, 'Santosh Hegde', 15),
 (10, 320, 'Prashant Bhushan', 21),
 (10, 323, 'Swami Agnivesh', 24),
 (12, 380, '‘ India Against Corruption', 3),
 (12, 400, 'Hazare', 23)]

In [83]:
vocab = ["strike",
"unrest",
"masses",
"protest",
"demonstration",
"worker",
"union",
"company",
"caste",
"religious",
"ethnic",
"reformed",
"rebellion",
"defense",
"violence",
"war",
"armed",
"fight",
"Right",
"free",
"freedom",
"liberty",
"justice",
"Fair",
"unfair",
"unequal",
"terror",
"extreme",
"Bomb",
"IED",
"weapon",
"gun",
"wmd",
"threat",
"suicide",
"murder",
"Kill",
"death",
"explosive",
"military",
"police",
"elite",
"government",
"oppresive",
"power",
"regime",
"fraud",
"corruption",
"coup",
"safety",
"secure",
"insecure",
"protect",
"enemy",
"resist",
"hostage",
"truce",
"fire",
"greed",
"panic",
"inflation",
"Price",
"Food",
"Water"]

In [84]:
pos_threshold = len(all_words)
all_whos = {k[2]:[] for k in whos}
#nearest_whos = {k[2]:[] for k in whos}
pos_occurences = {k[2]:[] for k in whos}
pos_scores = {k:-1 for k in pos_occurences.keys()}
unrest_scores = {k:-1 for k in all_whos.keys()}
for location in list(all_whos.keys()):
    sub_locations = location.split(' ')
    if len(sub_locations) > 1:
        last_subloc = sub_locations[-1]
        first_subloc = sub_locations[0]
        first_indices = [i for i, w in enumerate(all_words) if w == first_subloc]
        last_indices = [i for i, w in enumerate(all_words) if w == last_subloc]
        diff = [a_i - b_i for a_i, b_i in zip(last_indices, first_indices)]
        first_indices = first_indices[diff == len(sub_locations)]
        if not isinstance(first_indices,list):
            pos_occurences[location].append(first_indices)
        else:
            pos_occurences[location] = first_indices
        indices = pos_occurences[location]
    else:
        first_indices = [i for i, w in enumerate(all_words) if w == sub_locations[0]]
        if not isinstance(first_indices,list):
            pos_occurences[location].append(first_indices)
        else:
            pos_occurences[location] = first_indices
        indices = pos_occurences[location]
    print(location)
    print(indices)
    if len(indices)>0:
        no_of_indices = len(indices)
        i=0
        b=[]
        #nearest_whos = []
        while i< no_of_indices:
            j=0
            a = []
            l = []
            while j<pos_threshold and indices[i]+j<len(all_words):
                if all_words[indices[i]+j].lower() in vocab:
                    s = [j,indices[i],'r',all_words[indices[i]+j].lower()]
                    a.append(s)
                if all_words[indices[i]+j] in all_whos:
                    lo = [j,indices[i],'r',all_words[indices[i]+j].lower()]
                    l.append(lo)
                j=j+1
            j=0
            while j<pos_threshold and indices[i]-j>=0:
                if all_words[indices[i]-j].lower() in vocab:
                    s = [j,indices[i],'l',all_words[indices[i]-j].lower()]
                    a.append(s)
                if all_words[indices[i]-j] in all_whos:
                    lo = [j,indices[i],'l',all_words[indices[i]-j].lower()]
                    l.append(lo)
                j=j+1
            i=i+1
            b = b+a
            #nearest_whos = nearest_whos+l
        all_whos[location] = b
        #nearest_whos[location] = nearest_whos
#for i in pos_scores.keys():
#    a = []
#    for j in pos_occurences[i]:
#        if decay == 'linear':
#            a.append(linear_decay(j,len(all_words)))
#        else:
#            a.append(exp_decay(j,len(all_words)))
#    pos_scores[i] = a
#for i in unrest_scores.keys():
#    a = []
#    for j in all_whos[i]:
#        if decay == 'linear':
#            a.append(linear_decay(j[0],len(all_words)))
#        else:
#            a.append(exp_decay(j[0],len(all_words)))
#    unrest_scores[i] = a

Anna Hazare
[0]
Jan Lokpal Bill Veteran Gandhian
[7]
Jan Lokpal Bill
[7]
Mahatma Gandhi
[62]
Swami Agnivesh
[76]
Kiran Bedi
[85]
Sandeep Pandey
[92]
Hazare
[1, 15, 131, 148, 184, 203, 234, 237, 240, 296, 300, 400]
Prime Minister s Office
[44]
Bill
[9, 34, 127, 285]
Santosh Hegde
[314]
Prashant Bhushan
[320]
‘ India Against Corruption
[341]


In [85]:
unrest_whos = all_whos

In [86]:
#all_whos = sorted(all_whos, key=lambda x: len(x[2]),reverse=True)

In [103]:
#all_whos


In [88]:
unrest_whos

{'Anna Hazare': [[5, 0, 'r', 'death'],
  [21, 0, 'r', 'death'],
  [37, 0, 'r', 'corruption'],
  [101, 0, 'r', 'death'],
  [104, 0, 'r', 'government'],
  [263, 0, 'r', 'corruption'],
  [281, 0, 'r', 'government'],
  [309, 0, 'r', 'justice'],
  [332, 0, 'r', 'government'],
  [375, 0, 'r', 'protest'],
  [383, 0, 'r', 'corruption']],
 'Bill': [[12, 9, 'r', 'death'],
  [28, 9, 'r', 'corruption'],
  [92, 9, 'r', 'death'],
  [95, 9, 'r', 'government'],
  [254, 9, 'r', 'corruption'],
  [272, 9, 'r', 'government'],
  [300, 9, 'r', 'justice'],
  [323, 9, 'r', 'government'],
  [366, 9, 'r', 'protest'],
  [374, 9, 'r', 'corruption'],
  [4, 9, 'l', 'death'],
  [3, 34, 'r', 'corruption'],
  [67, 34, 'r', 'death'],
  [70, 34, 'r', 'government'],
  [229, 34, 'r', 'corruption'],
  [247, 34, 'r', 'government'],
  [275, 34, 'r', 'justice'],
  [298, 34, 'r', 'government'],
  [341, 34, 'r', 'protest'],
  [349, 34, 'r', 'corruption'],
  [13, 34, 'l', 'death'],
  [29, 34, 'l', 'death'],
  [136, 127, 'r', 'co

In [89]:
#sorted(unrest_whos, key=len, reverse=True)

In [90]:
all_whos = who_candidates

In [92]:
i=0;
whos = {}
while i<len(all_whos):
    if all_whos[i][2] not in whos:
        whos[all_whos[i][2]] = [(all_whos[i][0],all_whos[i][1],all_whos[i][3])]
    j=i+1;
    while j < len(all_whos):
        if all_whos[j][2].lower() == all_whos[i][2].lower():
            whos[all_whos[i][2]]+=[(all_whos[j][0],all_whos[j][1],all_whos[j][3])]
            all_whos.pop(j);
            #do not increment j
            continue;
        if findWholeWord(all_whos[j][2].lower())(all_whos[i][2].lower()) or findWholeWord(all_whos[i][2].lower())(all_whos[j][2].lower()):
            whos[all_whos[i][2]]+=[(all_whos[j][0],all_whos[j][1],all_whos[j][3])]
            all_whos.pop(j);
            continue;
        j+=1;
    i+=1

In [93]:
modified_who_candidates = whos

In [94]:
modified_who_candidates
#sentence_number, word_position in document, word_position in sentence.

{'Anna Hazare': [(0, 0, 0),
  (0, 14, 14),
  (3, 131, 36),
  (4, 148, 12),
  (5, 184, 21),
  (6, 203, 11),
  (7, 234, 24),
  (8, 237, 1),
  (8, 240, 4),
  (9, 296, 18),
  (10, 300, 1),
  (12, 400, 23)],
 'Jan Lokpal Bill Veteran Gandhian': [(0, 7, 7),
  (0, 32, 32),
  (3, 125, 30),
  (9, 285, 7)],
 'Kiran Bedi': [(2, 85, 18)],
 'Mahatma Gandhi': [(1, 62, 23)],
 'Prashant Bhushan': [(10, 320, 21)],
 'Prime Minister s Office': [(5, 168, 5)],
 'Sandeep Pandey': [(2, 92, 25)],
 'Santosh Hegde': [(10, 314, 15)],
 'Swami Agnivesh': [(2, 76, 9), (10, 323, 24)],
 '‘ India Against Corruption': [(12, 380, 3)]}

In [119]:
pos_scores = {k:0 for k in modified_who_candidates.keys()}
sent_scores = {k:0 for k in modified_who_candidates.keys()}
l = len(all_words)
n = len(text_sent)
for s in modified_who_candidates.keys():
    pos = modified_who_candidates[s];
    pos_score = 0;
    sent_score = 0;
    for p in pos:
        pos_score+=(1 - p[1]/l)
        sent_score+=(1 - p[0]/n)
    pos_score = pos_score/len(pos)
    sent_score = sent_score/len(pos)
    pos_scores[s] = pos_score
    sent_scores[s] = sent_score

In [121]:
sent_scores

{'Anna Hazare': 0.5384615384615384,
 'Jan Lokpal Bill Veteran Gandhian': 0.7692307692307692,
 'Kiran Bedi': 0.8461538461538461,
 'Mahatma Gandhi': 0.9230769230769231,
 'Prashant Bhushan': 0.23076923076923073,
 'Prime Minister s Office': 0.6153846153846154,
 'Sandeep Pandey': 0.8461538461538461,
 'Santosh Hegde': 0.23076923076923073,
 'Swami Agnivesh': 0.5384615384615384,
 '‘ India Against Corruption': 0.07692307692307687}

In [ ]:
#unrest_whos['Anna Hazare'].extend(unrest_whos['Hazare'])

In [ ]:
#unrest_whos['Anna Hazare']

In [95]:
all_whos = unrest_whos
i=0;
whos = {}
who_keys = sorted(unrest_whos, key=len, reverse=True)
while i<len(who_keys):
    if who_keys[i] not in whos:
        print(who_keys[i])
        print('-----------')
        whos[who_keys[i]] = all_whos[who_keys[i]]
    j=i+1;
    #t = [];
    while j < len(who_keys):
        #print(who_keys[i])
        if who_keys[i].lower() == who_keys[j].lower():
            whos[who_keys[i]].extend(all_whos[who_keys[j]])
            who_keys.pop(j);
            #do not increment j
            continue;
        if findWholeWord(who_keys[i].lower())(who_keys[j].lower()) or findWholeWord(who_keys[j].lower())(who_keys[i].lower()):
            whos[who_keys[i]].extend(all_whos[who_keys[j]])
            print(who_keys[i])
            print(who_keys[j])
            who_keys.pop(j);
            continue;
        j+=1;
    #if(len(t) !=0):
    #    print('adding....')
    #print(t)
    #whos[who_keys[i]].extend(t)
    i+=1
modified_unrest_whos = whos

Jan Lokpal Bill Veteran Gandhian
-----------
Jan Lokpal Bill Veteran Gandhian
Jan Lokpal Bill
Jan Lokpal Bill Veteran Gandhian
Bill
‘ India Against Corruption
-----------
Prime Minister s Office
-----------
Prashant Bhushan
-----------
Mahatma Gandhi
-----------
Swami Agnivesh
-----------
Sandeep Pandey
-----------
Santosh Hegde
-----------
Anna Hazare
-----------
Anna Hazare
Hazare
Kiran Bedi
-----------


In [96]:
modified_unrest_whos

{'Anna Hazare': [[5, 0, 'r', 'death'],
  [21, 0, 'r', 'death'],
  [37, 0, 'r', 'corruption'],
  [101, 0, 'r', 'death'],
  [104, 0, 'r', 'government'],
  [263, 0, 'r', 'corruption'],
  [281, 0, 'r', 'government'],
  [309, 0, 'r', 'justice'],
  [332, 0, 'r', 'government'],
  [375, 0, 'r', 'protest'],
  [383, 0, 'r', 'corruption'],
  [4, 1, 'r', 'death'],
  [20, 1, 'r', 'death'],
  [36, 1, 'r', 'corruption'],
  [100, 1, 'r', 'death'],
  [103, 1, 'r', 'government'],
  [262, 1, 'r', 'corruption'],
  [280, 1, 'r', 'government'],
  [308, 1, 'r', 'justice'],
  [331, 1, 'r', 'government'],
  [374, 1, 'r', 'protest'],
  [382, 1, 'r', 'corruption'],
  [6, 15, 'r', 'death'],
  [22, 15, 'r', 'corruption'],
  [86, 15, 'r', 'death'],
  [89, 15, 'r', 'government'],
  [248, 15, 'r', 'corruption'],
  [266, 15, 'r', 'government'],
  [294, 15, 'r', 'justice'],
  [317, 15, 'r', 'government'],
  [360, 15, 'r', 'protest'],
  [368, 15, 'r', 'corruption'],
  [10, 15, 'l', 'death'],
  [132, 131, 'r', 'corruptio

In [ ]:
whos

In [ ]:
title_score = {k:0 for k in whos.keys()}

In [ ]:
title = "Anna Hazare on fast unto death demanding Jan Lokpal Bill"

In [91]:
def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

In [ ]:
title_words = nltk.word_tokenize(title)
for k in title_score.keys():
    for w in title_words:
        if k.lower() == w.lower():
            title_score[k] = 1;
        if findWholeWord(k.lower())(w.lower()) or findWholeWord(w.lower())(k.lower()):
            print(k)
            print(w)
            title_score[k] = 1;

In [ ]:
title_score

In [126]:
thresh = 10;
unrest_scores = {k:-1 for k in modified_unrest_whos.keys()}
for l in modified_unrest_whos.keys():
    score =0;
    if len(modified_unrest_whos[l]) >0:
        locs_unrest = modified_unrest_whos[l]
        for l2 in locs_unrest:
            if l2[0]<thresh:
                score = score+(1/l2[0]);  
            else:
                score = score+np.exp(-l2[0]); 
    unrest_scores[l] = score/len(modified_unrest_whos);

In [127]:
unrest_scores

{'Anna Hazare': 0.07278257941636589,
 'Jan Lokpal Bill Veteran Gandhian': 0.1833343401358091,
 'Kiran Bedi': 1.1813797115679782e-08,
 'Mahatma Gandhi': 1.3887957551022785e-12,
 'Prashant Bhushan': 2.2845913143585416e-06,
 'Prime Minister s Office': 0.014285714295976165,
 'Sandeep Pandey': 0.011111725532346444,
 'Santosh Hegde': 0.020000001522998443,
 'Swami Agnivesh': 1.4579395423883515e-12,
 '‘ India Against Corruption': 0.01111111111111255}

In [131]:
sent_nos = {k:[] for k in modified_who_candidates.keys()}
for k in modified_who_candidates.keys():
    a = modified_who_candidates[k]
    c = []
    for b in a:
        c.extend([b[0]])
    sent_nos[k] = set(c) 

In [132]:
sent_nos

{'Anna Hazare': {0, 3, 4, 5, 6, 7, 8, 9, 10, 12},
 'Jan Lokpal Bill Veteran Gandhian': {0, 3, 9},
 'Kiran Bedi': {2},
 'Mahatma Gandhi': {1},
 'Prashant Bhushan': {10},
 'Prime Minister s Office': {5},
 'Sandeep Pandey': {2},
 'Santosh Hegde': {10},
 'Swami Agnivesh': {2, 10},
 '‘ India Against Corruption': {12}}